In [3]:
import numpy as np
import pandas as pd
import scipy as sp
def fun(x):
    w = 1
    a = -0.5
    b = 0.1
    return np.array([-w*x[1] + x[0]*x[1] + (a*x[0] - b*x[1])*(x[0]**2 + x[1]**2) + (x[0]+x[1])**5,
                      w*x[0] - 2*x[0]**2 + (b*x[0] + a*x[1])*(x[0]**2 + x[1]**2) - (x[0]+x[1])**5])


def f1(x):
    w = 1
    a = -0.5
    b = 0.1
    return np.array([-w*x[1] + x[0]*x[1] + (a*x[0] - b*x[1])*(x[0]**2 + x[1]**2) + (x[0]+x[1])**5])


def f2(x):
    w = 1
    a = -0.5
    b = 0.1
    return np.array([w*x[0] - 2*x[0]**2 + (b*x[0] + a*x[1])*(x[0]**2 + x[1]**2) - (x[0]+x[1])**5])

def T2_mat(n):
    T2 = np.eye(n**2,dtype=int)
    rmidx = np.triu_indices(n,1)[1]*n + np.triu_indices(n,1)[0]
    T2 = np.delete(T2,rmidx,0)
    return T2


def S2_mat(n):
    S2 = np.eye(n**2,dtype=int)
    rmidx = np.triu_indices(n,1)[1]*n + np.triu_indices(n,1)[0]
    addidx = np.triu_indices(n,1)[0]*n + np.triu_indices(n,1)[1]
    S2[rmidx,addidx] = 1
    S2 = np.delete(S2,rmidx,1)
    return S2

def T3_mat(n):
    Bx3 = [(i,j,k) for i in range(n) for j in range(i,n) for k in range(j,n)] # extracted from x \otimes Bx^2
    x_Bx2 = [(i,j,k) for i in range(n) for j in range(n) for k in range(j,n)] #  x \otimes Bx^2
    Bx3_idx = [x_Bx2.index(i) for i in Bx3]
    rmidx = list(set(range(len(x_Bx2)))-set(Bx3_idx))
    rmele = [x_Bx2[i] for i in rmidx]
    rmele = [tuple(sorted(i)) for i in rmele]
    rmidx_inBx3 = [Bx3.index(i) for i in rmele]
    T3 = np.eye(n*n*(n+1)//2,dtype=int)
    T3 = T3[Bx3_idx]
    return T3

def S3_mat(n):
    Bx3 = [(i,j,k) for i in range(n) for j in range(i,n) for k in range(j,n)] # extracted from x \otimes Bx^2
    x_Bx2 = [(i,j,k) for i in range(n) for j in range(n) for k in range(j,n)] #  x \otimes Bx^2
    Bx3_idx = [x_Bx2.index(i) for i in Bx3]
    rmidx = list(set(range(len(x_Bx2)))-set(Bx3_idx))
    rmele = [x_Bx2[i] for i in rmidx]
    rmele = [tuple(sorted(i)) for i in rmele]
    rmidx_inBx3 = [Bx3.index(i) for i in rmele]
    S3 = np.eye(n*n*(n+1)//2,dtype=int)
    S3 = S3[:,Bx3_idx]
    S3[rmidx,rmidx_inBx3] = 1
    return S3

def Trissian(f_test, x0):
    """
    This function calculates the 3rd order derivative of a function f: R^n -> R
    input: 
        f_test is the function
        x0 where the 3rd order want to be calcuated
    return: 3-D matrix
    """
    Trissian = np.zeros((x0.shape[0],x0.shape[0],x0.shape[0]))
    for i in range(x0.shape[0]):
        h = 0.001
        xp1 = np.array(x0, copy=True) 
        xp1[i] += h
        #print(xp1)
        xp2 = np.array(x0, copy=True) 
        xp2[i] += 2*h
        #print(xp2)
        xm1 = np.array(x0, copy=True) 
        xm1[i] -= h
        #print(xm1)
        xm2 = np.array(x0, copy=True) 
        xm2[i] -= 2*h
        #print(xm2)
        Trissian[i] = (-nd.Hessian(f_test)(xp2) + 8*nd.Hessian(f_test)(xp1)- 8*nd.Hessian(f_test)(xm1) + nd.Hessian(f_test)(xm2))/(12*h)
    return Trissian

<font color='red'>$$\dot{u} = J_c u + U_u^{2} B_u^2 + U_{u}^3 B_u^3 + O(u^4)$$

In [4]:
import numdifftools as nd
x = np.array([0,0],dtype=float) #Should explicitly assign dtype
J_c = nd.Jacobian(fun)(x)
# J_c

In [5]:
n_c = 2
J_cbar = np.kron(np.eye(n_c),J_c) + np.array([np.kron(np.eye(n_c),row) for row in J_c]).reshape(-1,n_c**2)
# display(pd.DataFrame(J_cbar))
C2 = -np.matmul(np.matmul(T2_mat(n_c),J_cbar),S2_mat(n_c))
# display(pd.DataFrame(C2))

In [6]:
Uu2 = np.array([nd.Hessian(f1)(x)[np.triu_indices(n_c,0)],nd.Hessian(f2)(x)[np.triu_indices(n_c,0)]])
# Uu2
idx = [(i,j,k) for i in range(n_c) for j in range(i,n_c) for k in range(j,n_c)]
Uu3 = np.array([[Trissian(f1, x)[j] for j in idx],[Trissian(f2, x)[j] for j in idx]])

In [7]:
Uu2 = Uu2 * np.array([[0.5,1.0,0.5],[0.5,1.0,0.5]])

In [8]:
Uu3 = Uu3 * np.array([[1/6,1/2,1/2,1/6],[1/6,1/2,1/2,1/6]])

In [9]:
Uu2

array([[ 0.,  1.,  0.],
       [-2.,  0.,  0.]])

In [10]:
Uu3

array([[-0.5, -0.1, -0.5, -0.1],
       [ 0.1, -0.5,  0.1, -0.5]])

In [12]:
import scipy as sp
Lambda2 = np.kron(np.eye(n_c*(n_c+1)//2),J_c) + np.kron(C2.T, np.eye(n_c))
# Since we know 2nd order will be eliminated, so Lambda should be full rank
assert(np.linalg.matrix_rank(Lambda2) == Lambda2.shape[0])
# So R_2n will be zero and H_2n could be solved directly from the Sylvester Equation
H_2n = sp.linalg.solve_sylvester(J_c, C2, -Uu2)
#display(pd.DataFrame(H_2n))
R_2n = np.dot(J_c,H_2n) + np.dot(H_2n,C2) + Uu2
R_2n

array([[-4.85877091e-16,  8.88178420e-16, -4.89362087e-16],
       [-4.44089210e-16, -8.91498180e-16,  2.22044605e-16]])

In [13]:
H_bar = np.kron(np.eye(n_c),np.dot(H_2n,T2_mat(n_c))) + np.array([np.kron(np.eye(n_c),row) for row in np.dot(H_2n,T2_mat(n_c))]).reshape(-1,n_c**3)
#display(pd.DataFrame(H_bar))
R_bar = np.kron(np.eye(n_c),np.dot(R_2n,T2_mat(n_c))) + np.array([np.kron(np.eye(n_c),row) for row in np.dot(R_2n,T2_mat(n_c))]).reshape(-1,n_c**3)
#display(pd.DataFrame(R_bar))
Ny3 = np.linalg.multi_dot([Uu2,T2_mat(n_c),H_bar,np.kron(np.eye(n_c),S2_mat(n_c)),S3_mat(n_c)]) + Uu3 - np.linalg.multi_dot([H_2n,T2_mat(n_c),R_bar,np.kron(np.eye(n_c),S2_mat(n_c)),S3_mat(n_c)])
display(pd.DataFrame(Ny3))
J_bar = np.kron(np.eye(n_c**2),J_c) + np.kron(np.eye(n_c), np.array([np.kron(np.eye(n_c),row) for row in J_c]).reshape(-1,n_c**2)) + np.array([np.kron(np.eye(n_c**2),row) for row in J_c]).reshape(-1,n_c**3) 
#display(pd.DataFrame(J_bar))
C3 = -np.linalg.multi_dot([T3_mat(n_c),np.kron(np.eye(n_c),T2_mat(n_c)),J_bar,np.kron(np.eye(n_c),S2_mat(n_c)),S3_mat(n_c)])
#display(pd.DataFrame(C3))
# Lambda3 = np.kron(np.eye(n_c*(n_c+1)*(n_c+2)//6),J_c) + np.kron(C3.T, np.eye(n_c))
Lambda3 = np.kron(J_c, np.eye(n_c*(n_c+1)*(n_c+2)//6)) + np.kron(np.eye(n_c), C3.T) # This is row vec
#display(pd.DataFrame(Lambda3))

,0,1,2,3
0,-0.500000,-1.433333,-0.500000,1.566667
1,-1.233333,-0.500000,-6.566667,-0.500000


In [14]:
Lambda3

array([[ 0., -1.,  0.,  0., -1., -0., -0., -0.],
       [ 3.,  0., -2.,  0.,  0., -1., -0., -0.],
       [ 0.,  2.,  0., -3., -0.,  0., -1., -0.],
       [ 0.,  0.,  1.,  0., -0., -0.,  0., -1.],
       [ 1.,  0.,  0.,  0.,  0., -1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  3.,  0., -2.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  2.,  0., -3.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.]])

In [15]:
np.linalg.matrix_rank(Lambda3)

6

In [16]:
# display(pd.DataFrame(Lambda3))
P, L, U = sp.linalg.lu(Lambda3,permute_l=False)
L = P.dot(L)
# display(pd.DataFrame(L))
# display(pd.DataFrame(U))
# display(pd.DataFrame(L.dot(U)))
L_inv = np.linalg.inv(L)
#display(pd.DataFrame(L_inv))

L3 = np.array([[1,0,1,0,0,1,0,1],[0,-1,0,-1,1,0,1,0]]).T
L2 = L_inv[6:]

theta =np.linalg.inv(L2.dot(L3)).dot(L2).dot(Ny3.reshape((1,-1)).T)
theta

array([[-0.5       ],
       [-1.69166667]])

In [48]:
q, r = np.linalg.qr(Lambda3)
Ap = r[:,0:np.linalg.matrix_rank(Lambda3)].dot(-np.reshape(Ny3,-1).dot(r[:,0:6]))
H3n = np.linalg.solve(Lambda3, Ap)
R3n = -np.reshape(Ny3,-1) - Ap
R3n

array([ -6.13180943, -18.47015528,   1.83236682,  17.81078678,
        -4.32506124,  -1.37855816,   6.56666667,   0.5       ])

In [25]:
T = np.array([[1,1j],[1,-1j]],dtype=complex)
T_inv = np.linalg.inv(T)
B2 = np.linalg.multi_dot([T2_mat(2),np.kron(T_inv,T_inv),S2_mat(2)])
B3 = np.linalg.multi_dot([T3_mat(2),np.kron(np.eye(2),T2_mat(2)),np.kron(np.kron(T_inv,T_inv),T_inv),np.kron(np.eye(2),S2_mat(2)),S3_mat(2)])
display(pd.DataFrame(B2))
display(pd.DataFrame(B3))
display(pd.DataFrame(np.array([[1/6, 1/2, 1/2, 1/6],[1/6, 1/2, 1/2, 1/6]]).dot(B3)))

,0,1,2
0,(0.25+0j),(0.5+0j),(0.25+0j)
1,-0.25j,0j,0.25j
2,(-0.25+0j),(0.5+0j),(-0.25+0j)


,0,1,2,3
0,(0.125+0j),(0.375+0j),(0.375+0j),(0.125+0j)
1,-0.125j,-0.125j,0.125j,0.125j
2,(-0.125+0j),(0.125+0j),(0.125+0j),(-0.125+0j)
3,0.125j,-0.375j,0.375j,-0.125j


,0,1,2,3
0,(-0.04166666666666667-0.04166666666666667j),(0.125-0.125j),(0.125+0.125j),(-0.04166666666666667+0.04166666666666667j)
1,(-0.04166666666666667-0.04166666666666667j),(0.125-0.125j),(0.125+0.125j),(-0.04166666666666667+0.04166666666666667j)
